### 1 Preparação do ambiente

In [1]:
import platform
print('Plataforma usada no experimento')
print('='*40)

print('Jupyter Notebook no VSCode')
print(f'Processaor: {platform.processor()}')
print(f'SO: {platform.system()}')

Plataforma usada no experimento
Jupyter Notebook no VSCode
PRocessaor: Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
SO: Windows


In [2]:
%%capture
!git clone https://github.com/joerg84/ArandoDBUniversity.git
!rsync -av ArangoDBUniversity/ ./ --exclude.git
!pip3 install pyarango 
!pip3 install "python-arango>=5.0"

In [4]:
import json
import requests
import sys
#import oasis
import time 

from pyArango.connection import *
conn = Connection(username="root", password="1234")

### Criando e abrindo bancos de dados

Com uma conexão com o ArangoDB Server, você pode abrir ou criar um banco de dados no servidor e começar a operar nele. O `createDatabase()` método na conexão do servidor lida com as duas operações, retornando um `Database` instância.

In [5]:
db = conn.createDatabase(name="school")

Quando o `school` o banco de dados não existe, o pyArango o cria na conexão do servidor. Quando existe, ele tenta abrir o banco de dados. Você também pode abrir um banco de dados existente usando seu nome como uma chave na conexão do servidor. Por exemplo,

In [7]:
db = conn["school"]
db

ArangoDB database: school

### Criando coleções

O ArangoDB agrupa documentos e arestas em coleções. Isso é semelhante ao conceito de tabelas nos bancos de dados relacionais, mas com a principal diferença de que as coleções são sem esquema.

No pyArango, você pode criar uma coleção chamando o método `createCollection()` em um determinado banco de dados. Por exemplo, na seção acima, você criou um banco de dados `school`. Você pode querer uma coleção nesse banco de dados para os alunos.

In [9]:
studentsCollection = db.createCollection(name="Students")
db["Students"]

ArangoDB collection name: Students, id: 1885, type: document, status: loaded

### Criando documentos

Com o banco de dados e a configuração da coleção, você pode começar a adicionar dados a eles. Continuando a comparação com os bancos de dados relacionais, onde uma coleção é uma tabela, um documento é uma linha nessa tabela. Ao contrário das linhas, no entanto, os documentos são sem esquema. Você pode incluir qualquer arranjo de valores necessário para sua aplicação.

In [10]:
doc1 = studentsCollection.createDocument()
doc1["name"] = "John Smith"
doc1
doc2 = studentsCollection.createDocument()
doc2["firstname"] = "Emily"
doc2["lastname"] = "Bronte"
doc2

ArangoDoc '_id: None, _key: None, _rev: None': <store: {'firstname': 'Emily', 'lastname': 'Bronte'}>

O documento mostra sua `_id` como *“Nenhum”* porque você ainda não o salvou no ArangoDB. Isso significa que a variável existe no seu código Python, mas não no banco de dados. Construções ArangoDB `_id` valores combinando o nome da coleção com o `_key` valor. Ele também lida com a tarefa para você, você só precisa definir a chave e salvar o documento.

In [11]:
doc1._key = "johnsmith"
doc1.save()
doc1

ArangoDoc '_id: Students/johnsmith, _key: johnsmith, _rev: _f3qcavu---': <store: {'name': 'John Smith'}>

Em vez de inserir e salvar os dados para todos os alunos manualmente, convém inserir os dados por meio de um loop, em vez de chamadas individuais. Por exemplo,

In [14]:
students = [('Oscar', 'Wilde', 3.5), ('Thomas', 'Hobbes', 3.2), 
('Mark', 'Twain', 3.0), ('Kate', 'Chopin', 3.8), ('Fyodor', 'Dostoevsky', 3.1), 
('Jane', 'Austen',3.4), ('Mary', 'Wollstonecraft', 3.7), ('Percy', 'Shelley', 3.5), 
('William', 'Faulkner', 3.8), ('Charlotte', 'Bronte', 3.0)]
for (first, last, gpa) in students:
    doc = studentsCollection.createDocument()
    doc['name'] = "%s %s" % (first, last)
    doc['gpa'] = gpa 
    doc['year'] = 2017
    doc._key = ''.join([first, last]).lower() 
    doc.save()

UniqueConstrainViolation: unique constraint violated - in index primary of type primary over '_key'; conflicting key: oscarwilde. Errors: {'code': 409, 'error': True, 'errorMessage': "unique constraint violated - in index primary of type primary over '_key'; conflicting key: oscarwilde", 'errorNum': 1210}

### Leitura de documentos

Eventualmente, você precisará acessar documentos no banco de dados. A maneira mais fácil de fazer isso é com o `_key` valor.

Por exemplo, o banco de dados da escola agora tem vários alunos. Imagine-o como parte de um aplicativo maior, com mais dados disponíveis sobre cada aluno e você gostaria de verificar o GPA de um aluno em particular:

In [15]:
def report_gpa(document):
    print("Student: %s" % document['name'])
    print("GPA:     %s" % document['gpa'])
kate = studentsCollection['katechopin']
report_gpa(kate)

Student: Kate Chopin
GPA:     3.8


### Atualizando documentos

Ao ler um documento do ArangoDB em seu aplicativo, você cria uma cópia local do documento. Você pode operar o documento, fazendo as alterações que desejar e depois enviar os resultados para o banco de dados usando o método `save()`.

Por exemplo, a cada semestre, à medida que as notas finais são provenientes de suas aulas, você precisa atualizar as médias de notas dos alunos no banco de dados. Como isso acontece com frequência, convém criar uma função específica para lidar com a atualização:

In [16]:
def update_gpa(key, new_gpa):
    doc = studentsCollection[key]
    doc['gpa'] = new_gpa
    doc.save()